# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nishinoomote,30.7333,131.0000,19.51,69,3,8.49,JP,1715110799
1,1,tazovsky,67.4667,78.7000,-13.28,97,93,3.81,RU,1715111183
2,2,papao,-17.7500,-149.5500,28.82,75,98,8.38,PF,1715111183
3,3,petropavlovsk-kamchatsky,53.0452,158.6483,0.81,78,100,3.58,RU,1715111183
4,4,port alfred,-33.5906,26.8910,21.07,93,26,7.25,ZA,1715111183


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot

city_map = city_data_df.hvplot.points("Lng","Lat", geo = True, tiles = "OSM", frame_width = 800, frame_height = 600, size = "Humidity",
                                          alpha = 0.5, color = 'City')

# Display the map

city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values

# A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

# Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

# Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
139,139,turabah,21.2141,41.6331,25.96,20,0,2.69,SA,1715111204
209,209,san sebastian de la gomera,28.0916,-17.1133,22.43,78,0,3.09,ES,1715111215
224,224,nsanje,-16.9200,35.2620,21.16,80,0,1.43,MW,1715111217
245,245,jamestown,42.0970,-79.2353,22.80,46,0,0.00,US,1715111220
302,302,karratha,-20.7377,116.8463,24.55,69,0,2.04,AU,1715111229
326,326,al badari,26.9926,31.4155,24.08,16,0,3.89,EG,1715111233
341,341,tsiombe,-25.3000,45.4833,21.04,85,0,3.20,MG,1715111235
381,381,hotan,37.0997,79.9269,21.02,24,0,1.63,CN,1715111242
428,428,mermoz boabab,14.7065,-17.4758,26.31,83,0,3.09,SN,1715111250
461,461,bukhara,39.7747,64.4286,24.97,41,0,3.09,UZ,1715111255


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
139,turabah,SA,21.2141,41.6331,20,
209,san sebastian de la gomera,ES,28.0916,-17.1133,78,
224,nsanje,MW,-16.9200,35.2620,80,
245,jamestown,US,42.0970,-79.2353,46,
302,karratha,AU,-20.7377,116.8463,69,
326,al badari,EG,26.9926,31.4155,16,
341,tsiombe,MG,-25.3000,45.4833,85,
381,hotan,CN,37.0997,79.9269,24,
428,mermoz boabab,SN,14.7065,-17.4758,83,
461,bukhara,UZ,39.7747,64.4286,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame

    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
   
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
turabah - nearest hotel: No hotel found
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
nsanje - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
karratha - nearest hotel: Karratha International Hotel
al badari - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
mermoz boabab - nearest hotel: La maison blanche
bukhara - nearest hotel: Odilnur
laguna larga - nearest hotel: No hotel found
port saint john's - nearest hotel: Outback Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
139,turabah,SA,21.2141,41.6331,20,No hotel found
209,san sebastian de la gomera,ES,28.0916,-17.1133,78,Hotel Villa Gomera
224,nsanje,MW,-16.9200,35.2620,80,No hotel found
245,jamestown,US,42.0970,-79.2353,46,DoubleTree Jamestown
302,karratha,AU,-20.7377,116.8463,69,Karratha International Hotel
326,al badari,EG,26.9926,31.4155,16,No hotel found
341,tsiombe,MG,-25.3000,45.4833,85,No hotel found
381,hotan,CN,37.0997,79.9269,24,Hua Yi International Hotel (accepts foreigners)
428,mermoz boabab,SN,14.7065,-17.4758,83,La maison blanche
461,bukhara,UZ,39.7747,64.4286,41,Odilnur


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
city_map  = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 800, frame_height = 600, size = "Humidity",
                                   color="City", alpha=.75, hover_cols=["Hotel Name", "Country"])

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)